Сейчас я попробую классифицировать тексты не на тех признаках, которые я сама выделила, а на тех, которые вернет КаунтВекторайзер

In [1]:
import re
import nltk
import numpy as np
import pandas as pd
from nltk import word_tokenize
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, homogeneity_score, completeness_score, v_measure_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB


In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lera\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
df = pd.read_csv('gk4z_texts.csv')
df.head()

,id,text,likes,label
0,76000,#конкурс@the4gkz \r\nДля вашего удобства собра...,17,post
1,76914,Уже сегодня ночью (по МСК) начнётся один из са...,34,post
2,76917,"Ого, Нина Кравиц в лайн-апе😲",0,comment
3,76920,"[id17823823|Анастасия], с новым крутым шоу от ...",0,comment_to_comment
4,76922,Скину расписание на сегодня сюда. Время москов...,0,comment


In [5]:
df.text.fillna('', inplace=True) # заменяем нан на пустую строку

##### Сначала попробую все основные классификаторы и посмотрю, какие результаты. 
В качестве токенизатора взяла punkt_tokenizer, потому что я считаю, что здесь очень важно хорошее распознавание знаков препинания.

Intuitively, precision is the ability of the classifier not to label as positive a sample that is negative, and recall is the ability of the classifier to find all the positive samples. The F-measure can be interpreted as a weighted harmonic mean of the precision and recall.

In [6]:
vectorizer = CountVectorizer(tokenizer=wordpunct_tokenize)
bow = vectorizer.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bow, df.label)

In [7]:
bow

<56200x68660 sparse matrix of type '<class 'numpy.int64'>'
	with 674699 stored elements in Compressed Sparse Row format>

In [8]:
X_train

<42150x68660 sparse matrix of type '<class 'numpy.int64'>'
	with 507138 stored elements in Compressed Sparse Row format>

##### LogisticRegression

In [24]:
lr = LogisticRegression()
clf = lr.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

                    precision    recall  f1-score   support

           comment       0.87      0.94      0.90      9483
comment_to_comment       0.65      0.43      0.52      1165
              post       0.88      0.80      0.84      3402

       avg / total       0.85      0.86      0.85     14050



Интересно, что результат не хуже, чем по признакам, которые я сама составляла, а для комментов второго уровня даже лучше

##### TruncatedSVD + LogisticRegression

In [15]:
svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
svd.fit(bow)

TruncatedSVD(algorithm='randomized', n_components=50, n_iter=7,
       random_state=42, tol=0.0)

In [16]:
bow_svd = svd.transform(bow)
X_train_svd, X_test_svd, y_train_svd, y_test_svd = train_test_split(bow_svd, df.label)

In [12]:
bow_svd.shape

(56200, 500)

In [14]:
X_train_svd.shape


(42150, 500)

In [17]:
clf_svd = LogisticRegression()
clf_svd.fit(X_train_svd, y_train_svd)
print(classification_report(y_test_svd, clf_svd.predict(X_test_svd)))

                    precision    recall  f1-score   support

           comment       0.79      0.95      0.86      9469
comment_to_comment       0.41      0.08      0.14      1114
              post       0.84      0.61      0.71      3467

       avg / total       0.77      0.79      0.77     14050



##### Наивный Баес

In [8]:
nb = MultinomialNB()
nbc = nb.fit(X_train, y_train)
print(classification_report(y_test, nbc.predict(X_test)))

                    precision    recall  f1-score   support

           comment       0.82      0.91      0.86      9327
comment_to_comment       0.56      0.27      0.36      1228
              post       0.79      0.70      0.75      3495

       avg / total       0.79      0.80      0.79     14050



##### TfidfVectorizer + LogisticRegression

In [9]:
vectorizer2 = TfidfVectorizer(tokenizer=wordpunct_tokenize)
bow2 = vectorizer2.fit_transform(df.text)
X_train2, X_test2, y_train2, y_test2 = train_test_split(bow2, df.label)

In [10]:
lr2 = LogisticRegression()
clf2 = lr2.fit(X_train2, y_train2)
print(classification_report(y_test2, clf2.predict(X_test2)))

                    precision    recall  f1-score   support

           comment       0.84      0.94      0.89      9534
comment_to_comment       0.56      0.24      0.33      1113
              post       0.88      0.73      0.80      3403

       avg / total       0.83      0.84      0.82     14050



##### TfidfVectorizer + MultinomialNB

In [11]:
nb2 = MultinomialNB()
nbc2 = nb2.fit(X_train2, y_train2)
print(classification_report(y_test2, nbc2.predict(X_test2)))

                    precision    recall  f1-score   support

           comment       0.73      0.99      0.84      9534
comment_to_comment       1.00      0.00      0.00      1113
              post       0.94      0.29      0.44      3403

       avg / total       0.80      0.74      0.68     14050



здесть precision у комментов второго уровня единица, а всё остальное по нулям, видимо, классификатор ничего не записал в этот класс

##### Кластеризация

Посмотрю, как сработает кластеризатор. 

In [15]:
model = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, n_jobs=-1, random_state=44)
model.fit(bow)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=-1, precompute_distances='auto',
    random_state=44, tol=0.0001, verbose=0)

In [16]:
model2 = KMeans(n_clusters=3, init='k-means++', max_iter=300, n_init=10, n_jobs=-1, random_state=88)
model2.fit(bow2)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=-1, precompute_distances='auto',
    random_state=88, tol=0.0001, verbose=0)

In [17]:
adjusted_rand_score(model.labels_, model2.labels_)

0.1289879130768284

ага, кластеризатор не нашел какого-то очевидного параметра, как делить датасет. Есть ли совпадение с делением на посты\комменты?

In [21]:
print(adjusted_rand_score(model.labels_, df.label))
print(adjusted_rand_score(model2.labels_, df.label))

0.0229503766657702
0.014719670659369728


нет, никакого. Посмотрим на кластеры, будет ли прослеживаться какая-то логика

In [22]:
from sklearn.metrics import pairwise_distances_argmin_min
closest, _ = pairwise_distances_argmin_min(model.cluster_centers_, bow)

In [23]:
closest, _

(array([ 7895, 10713,    21], dtype=int64),
 array([ 5.70361734, 24.413762  ,  0.74364699]))

In [41]:
ultimate_texts = df.loc[[7895, 10713,    21]]
ultimate_texts
print(ultimate_texts.text[7895])
print(ultimate_texts.text[10713])
print(ultimate_texts.text[21])

друзья-москвичи, напоминаю, что сегодня во всем городе минимум школьников, поэтому все идем гулять в центр, на каток, в кино и т.д. и наслаждаться спокойствием!
#bestof2018@the4gkz
Продолжаем погружаться в мой мир песенок и открытий этого года. 
Данный список состоит из артистов, которые не вошли по каким-либо причинам в основной топ, но для меня их релизы были важны. 
 
1. СБПЧ – «Мы не спали, мы снились» 
Как и со многими группами, я пытался послушать ребят еще несколько лет назад, но что-то все как-то нет. И вот с релизом нового альбома я прямо-таки открыл для себя группу заново. Кирилл говорит, что под новый альбом нужно танцевать, а не вслушиваться в текста, но я так не согласен, у данного релиза просто потрясающие текста в песенках. Если вы все ждали момент, чтобы начать слушать эту группу, то сейчас самое время. 
 
2. Nothing But Thieves – “What Did You Think When You Made Me This Way?” 
Ну, тут даже и говорить ни о чем не нужно. Группа продолжает взлетать все выше и выше, выпус

In [53]:
df['clasters'] = model.labels_
df['clasters2'] = model2.labels_

df[df.clasters2==2]

,id,text,likes,label,clasters,clasters2
0,76000,#конкурс@the4gkz \r\nДля вашего удобства собра...,17,post,0,2
1,76914,Уже сегодня ночью (по МСК) начнётся один из са...,34,post,0,2
2,76917,"Ого, Нина Кравиц в лайн-апе😲",0,comment,2,2
4,76922,Скину расписание на сегодня сюда. Время москов...,0,comment,0,2
5,76933,"[id18399319|Anastasia], неправда, в Москве вре...",0,comment_to_comment,2,2
6,76929,"Трансляция, конечно, моё почтение. Классно выг...",1,comment,2,2
7,76930,"Моё увожение Блэкпинк, которые даже с живыми и...",1,comment,2,2
8,76934,"Часть пропустила, конец сета у SG Lewis был кл...",0,comment,0,2
9,76935,"сейчас посмотрела лайв Rufus Du Sol, очень бод...",0,comment,2,2
11,76908,"#newalbum@the4gkz \r\nКак обычно, рассказываем...",14,post,0,2


Судя по тому, что я увидела, в обоих случаях деление на кластеры произошло по признакам размера поста и наличия в нем латиницы.
Посмотрю центроиды в тф-идф

In [51]:
print("Top terms per cluster:")
order_centroids = model2.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer2.get_feature_names()
for i in range(3):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :20]:
        print(' %s' % terms[ind]),

Top terms per cluster:
Cluster 0:
 в
 на
 |
 [
 ],
 и
 не
 ?
 как
 что
 а
 -
 это
 я
 с
 ли
 у
 когда
 или
 .
Cluster 1:
 the4gkz
 #
 @
 eveningclassics
 -
 —
 newconcert
 (
 .
 )
 |
 москва
 ]
 the
 07
 [
 петербург
 санкт
 06
 alternativerock
Cluster 2:
 ,
 .
 не
 и
 в
 -
 |
 на
 [
 что
 ],
 я
 а
 то
 это
 но
 с
 так
 )
 !


Так себе центроиды, надо сказать. Кластер 0 и кластер 2 практически совпадают, и вошло в них то, что обычно является стоп-словами.По какому принципу он поделил их тогда, интересно? В класс 1 вошла латиница, это видно.

In [54]:
closest, _ = pairwise_distances_argmin_min(model2.cluster_centers_, bow2)
closest, _

(array([21, 21, 21], dtype=int64), array([0.07169889, 0.54878398, 0.21185283]))

In [55]:
df.text[21]

''

Ага. Вау. Плохо кластеризатор их поделил, вот что) Все три класса ближе всего к отсутствию текста. Результат понятный, учитывая то, что деления по тематике в датасете нет как такового, стоп-слова не удалены и вообще везде пишут, что тф-идф векторизация плохо работает на маленьких текстах, а в датасете их большинство.

##### CountVectorizer(analyzer="char", ngram_range=(2,2)) + LogisticRegression
проанализирую по символьным биграммам.

In [12]:
vectorizer3 = CountVectorizer(analyzer="char", ngram_range=(2,2))
bow3 = vectorizer3.fit_transform(df.text)

In [13]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(bow3, df.label)

In [14]:
lr3 = LogisticRegression()
clf3 = lr3.fit(X_train3, y_train3)
print(classification_report(y_test3, clf3.predict(X_test3)))

                    precision    recall  f1-score   support

           comment       0.86      0.92      0.89      9350
comment_to_comment       0.57      0.37      0.45      1195
              post       0.87      0.81      0.84      3505

       avg / total       0.84      0.85      0.84     14050



##### CountVectorizer(analyzer="char", ngram_range=(2,2)) + MultinomialNB()

In [15]:
nb3 = MultinomialNB()
nbc3 = nb3.fit(X_train3, y_train3)
print(classification_report(y_test3, nbc3.predict(X_test3)))

                    precision    recall  f1-score   support

           comment       0.77      0.76      0.76      9350
comment_to_comment       0.41      0.69      0.51      1195
              post       0.60      0.49      0.54      3505

       avg / total       0.70      0.68      0.68     14050



##### CountVectorizer(analyzer="char", ngram_range=(2,2)) + LogisticRegression + TruncatedSVD


In [16]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
svd = TruncatedSVD(n_components=500, n_iter=7, random_state=42)

In [21]:
svd.fit(bow3)

TruncatedSVD(algorithm='randomized', n_components=500, n_iter=7,
       random_state=42, tol=0.0)

In [22]:
bow_svd = svd.transform(bow3)
X_train_svd, X_test_svd, y_train_svd, y_test_svd = train_test_split(bow_svd, df.label)
clf_svd = LogisticRegression()
clf_svd.fit(X_train_svd, y_train_svd)
print(classification_report(y_test_svd, clf_svd.predict(X_test_svd)))

                    precision    recall  f1-score   support

           comment       0.84      0.92      0.88      9484
comment_to_comment       0.48      0.25      0.33      1145
              post       0.85      0.77      0.81      3421

       avg / total       0.81      0.83      0.82     14050



#### В итоге на признаках из КаунтВекторайзера самый высокий результат показала логистическая регрессия,
как на токенах, так и на символьных биграммах